# Transformer-based Natural Language Processing
## Introduction to 🤗 Transformers
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/texttechnologylab/WiSe23-TFb-NLP/blob/master/assignment.ipynb)

### Installing necessary packages (i.e. if on Colab)

In [ ]:
% pip install torch datasets tokenizers transformers

### Premise

This notebook will guide you through the process of finetuning a transformer model using the 🤗 Transformers library.

First, we need to select a task and suitable dataset. Here, we will use the [Textual Entailment or Natrual Language Inference](https://cims.nyu.edu/~sbowman/multinli/) task as an example. A suitable dataset can be found in the [GLUE repository on the 🤗 Hub](https://huggingface.co/datasets/glue). The whole MNLI dataset ist way too big, so we will only use a slice of it.

We can load the MNLI (slice) of the GLUE dataset using [🤗 Datasets](https://huggingface.co/docs/datasets/index) as follows:

In [1]:
from datasets import load_dataset

mnli_dataset = load_dataset("glue", "mnli", split={"train": "train[:10%]", "validation": "validation_matched", "test": "test_matched"})
print(mnli_dataset)

DatasetDict({
    train: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx'],
        num_rows: 39270
    })
    validation: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx'],
        num_rows: 9815
    })
    test: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx'],
        num_rows: 9796
    })
})


As we can see above, the dataset is already split into train, development and test splits.
Each row contains four, but we only need to focus the premise, hypothesis and the label.

The textual entailment task requires us to recognize, given two text fragments, whether the meaning of one text is entailed (*can be inferred*) from the other text.

In this example, we will use a BERT-family model. With BERT, we formulate the entailment task as a simple classification task by concatenating the premise and hypothesis and training our classifier on the first token (the `[CLS]` token) of the input string:

```
"[CLS] This is the premise, i.e. a text that means something. [SEP] This is the hypothesis, i.e. what we may be able to infer [SEP]"
```

But let's first take a look at the dataset.

In [2]:
print(mnli_dataset['train'][:2])
print(mnli_dataset['validation'][:2])
print(mnli_dataset['test'][:2])

{'premise': ['Conceptually cream skimming has two basic dimensions - product and geography.', 'you know during the season and i guess at at your level uh you lose them to the next level if if they decide to recall the the parent team the Braves decide to call to recall a guy from triple A then a double A guy goes up to replace him and a single A guy goes up to replace him'], 'hypothesis': ['Product and geography are what make cream skimming work. ', 'You lose the things to the following level if the people recall.'], 'label': [1, 0], 'idx': [0, 1]}
{'premise': ['The new rights are nice enough', 'This site includes a list of all award winners and a searchable database of Government Executive articles.'], 'hypothesis': ['Everyone really likes the newest benefits ', 'The Government Executive articles housed on the website are not able to be searched.'], 'label': [1, 2], 'idx': [0, 1]}
{'premise': ['Hierbas, ans seco, ans dulce, and frigola are just a few names worth keeping a look-out for

As we see above, the `test_matched` split contains **unlabeled** samples, be we can ignore that for now.

Let's construct the sentences as we outlined above.

*Note:* The `[CLS]` and final `[SEP]` will be added by the BERT's tokenizer, so we omit them here.

In [3]:
prepared_dataset = mnli_dataset.map(
    lambda sample: {'input': f"{sample['premise']} [SEP] {sample['hypothesis']}"},
    remove_columns=['premise', 'hypothesis', 'idx']
)
print(prepared_dataset['train'][:2])

Map:   0%|          | 0/39270 [00:00<?, ? examples/s]

{'label': [1, 0], 'input': ['Conceptually cream skimming has two basic dimensions - product and geography. [SEP] Product and geography are what make cream skimming work. ', 'you know during the season and i guess at at your level uh you lose them to the next level if if they decide to recall the the parent team the Braves decide to call to recall a guy from triple A then a double A guy goes up to replace him and a single A guy goes up to replace him [SEP] You lose the things to the following level if the people recall.']}


### Loading Pre-Trained Models

Now we need to load a pre-trained BERT model. You should use a subclass of [AutoModel](https://huggingface.co/docs/transformers/main/en/autoclass_tutorial).

#### Load and instantiate a model for the textual entailment task

In [ ]:
from transformers import AutoConfig, AutoTokenizer  #, AutoModelFor?

config = ...  # TODO
tokenizer = ...  # TODO
model = ...  # TODO

Now we could use the [Trainer](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.Trainer) class for easy training. You can follow the tutorial from [the official documentation](https://huggingface.co/docs/transformers/quicktour#trainer-a-pytorch-optimized-training-loop).

#### Write the training procedure

In [ ]:
from transformers import Trainer

trainer = Trainer(...)

# TODO

trainer.train()

# TODO: evaluation

### Custom Training
While using the trainer class is very convenient, if you have to run custom procedures during training, a regular training loop can be more accessible.

We do need to do our own tokenization, though.

In [ ]:
def encode_pt(batch: dict):
    return tokenizer(
        batch['input'],
        add_special_tokens=True,
        return_token_type_ids=False,
        return_attention_mask=False,
        padding=False,
        truncation=True,
    )


pt_dataset = prepared_dataset.map(encode_pt)
print(pt_dataset['train'][:2])

However, in a manual training loop, we will want to make use of PyTorch's DataLoaders, which require some extra care to collate batches with samples of different lengths.

#### Implement `custom_collate`:
- Pad and stack the `input_ids` in a tensor.
- Stack the labels in a tensor of type `long`.

In [ ]:
import torch
from torch.nn.utils.rnn import pad_sequence


def custom_collate(batch: list[dict]) -> tuple[torch.Tensor, torch.Tensor]:
    input_ids = ...
    label = ...
    return input_ids, label

#### Write the training and evaluation loops

In [ ]:
from tqdm.notebook import tqdm, trange
from torch.utils.data import DataLoader
from torch.optim import *  # TODO
from torch.nn import *  # TODO

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

criterion = ...  # TODO
optimizer = ...  # TODO
num_epochs = ...  # TODO
batch_size = ...  # TODO

train_dataloader = DataLoader(pt_dataset['train'], batch_size=batch_size, shuffle=True, collate_fn=custom_collate)
dev_dataloader = DataLoader(pt_dataset['validation'], batch_size=batch_size, shuffle=False, collate_fn=custom_collate)

model.to(device)
for epoch in trange(num_epochs, position=0):
    model.train()
    for batch in tqdm(train_dataloader, position=1, leave=False):
        ...  # TODO

    model.eval()
    for batch in tqdm(dev_dataloader, position=1, leave=False):
        ...  # TODO